# Домашнее задание № 11. Генерация текста

### Задание 1 (10 баллов).

В семинаре мы работали с датасетами инструкций alpaca и dolly. Они англоязычные. В домашке вам нужно создать аналогичный датасет на русском языке и обучить аналогичную модель на этом датасете.
В качестве итогового результата у вас должна получится модель, которая может связно отвечать на русскоязычные инструкции на русском языке. Приведите как минимум три разных примера. Правильность ответов не так важна, так как вы скорее всего будете использовать небольшие модели, но текст должен быть не рандомным.

Русскоязычный датасет инструкций должен быть больше 5 тысяч примеров. Он может быть основнован на alpaca/dolly (например, вы можете просто прогнать все через переводную модель, которая была на семинаре, или даже google translate). Или вы можете придумать способ создать аналогичный датасет каким-то другим способом (переделать открытые датасеты с помощью правил). Датасет может быть не уникальным, можно скооперироваться с одногруппниками и сделать один датасет на всех.

Вы можете попробовать дообучать любую небольшую decoder-only модель. Скорее всего лучше всего будут работать модели, изначально обученные на русском языке (rugpt например). Но возможно даже модели вроде opt можно будет дообучить на русскоязычных инструкциях.

Это задание гораздно менее определенное, по сравнению с предыдущими. Поэтому не стесняйтесь задавать дополнительные вопросы в чате или лично, если у вас возникнут трудности.

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install datasets
!pip install zstandard jsonlines

In [ ]:
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer

In [ ]:
import accelerate

In [ ]:
from datasets import load_dataset

data_alpaca = load_dataset("IlyaGusev/ru_turbo_alpaca")

#dataset = load_dataset("IlyaGusev/ru_turbo_saiga")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for IlyaGusev/ru_turbo_alpaca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/ru_turbo_alpaca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `d

In [ ]:
data_alpaca['train'][:3]

{'instruction': ['Опишите, как сделать горшок из глины.',
  'Опиши процесс изготовления торта. Укажи ингредиенты, необходимые для этого.',
  'Напиши аналог данной пословицы на современный лад: "Не говори гоп, пока не перепрыгнешь".'],
 'input': ['', '', ''],
 'output': ['Для изготовления горшка из глины сначала необходимо взять глину и размешать ее водой до состояния пластилина. Затем нужно сформировать глиняный комок нужной формы и тщательно откачать влагу из глины, чтобы она стала твердой. Горшок нужно обжечь в духовке при высокой температуре, чтобы он стал прочным и непроницаемым.',
  'Для изготовления торта необходимы следующие ингредиенты: мука, сахар, яйца, молоко, сливочное масло, разрыхлитель, ванилин. Сначала нужно смешать муку с сахаром и добавить яйца, молоко, разрыхлитель и ваниль. Затем тесто надо хорошо перемешать и разделить на несколько частей. Каждую часть нужно запекать в духовке до готовности. После этого торт надо собрать, сложив слои пирога вместе с начинкой и укра

In [ ]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [ ]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [ ]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [ ]:
# class SupervisedDataset(Dataset):
#     """Dataset for supervised fine-tuning."""


#     def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
#         super(SupervisedDataset, self).__init__()
#         logging.warning("Loading data...")
#         list_data_dict = json.load(open(data_path))

#         logging.warning("Formatting inputs...")
#         prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
#         sources = [
#             prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
#             for example in list_data_dict
#         ]
#         targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

#         logging.warning("Tokenizing inputs... This may take some time...")
#         data_dict = preprocess(sources, targets, tokenizer)

#         self.input_ids = data_dict["input_ids"]
#         self.labels = data_dict["labels"]

#     def __len__(self):
#         return len(self.input_ids)

#     def __getitem__(self, i) -> Dict[str, torch.Tensor]:
#         return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [ ]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""


    def __init__(self, data, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = data["train"]

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [ ]:
# model_name = 'facebook/opt-350m'
model_name = "facebook/opt-125m"
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        max_length=512,
        cache_dir="huggingface_cache",
    )

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="huggingface_cache",
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

In [ ]:
train_dataset = SupervisedDataset(tokenizer=tokenizer, data=data_alpaca)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [ ]:
train_args = transformers.TrainingArguments(learning_rate=1e-5,
                 num_train_epochs=1,
                 per_device_train_batch_size=2,
                 gradient_accumulation_steps=1,
                 evaluation_strategy='no',
                 weight_decay=0.,
                 warmup_ratio=0.03,
                 lr_scheduler_type="cosine",
                 save_strategy='no',
                 logging_steps=1000,
                 output_dir="opt125_instruct_ft")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(model=model,
                 tokenizer=tokenizer,
                 args=train_args,
                 train_dataset=train_dataset,
                 eval_dataset=None,
                 data_collator=data_collator)

In [ ]:
trainer.train()

Step,Training Loss
1000,1.268900
2000,1.176900
3000,1.114800
4000,1.095700
5000,1.073800
6000,1.045500
7000,1.032300
8000,1.017200
9000,1.001100
10000,0.993500


TrainOutput(global_step=14911, training_loss=1.0486835374589754, metrics={'train_runtime': 4257.5852, 'train_samples_per_second': 7.004, 'train_steps_per_second': 3.502, 'total_flos': 6836609053440000.0, 'train_loss': 1.0486835374589754, 'epoch': 1.0})

In [ ]:
trainer.save_model('opt125_ft_02')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
MODEL_NAME = 'opt125_ft_02'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512, max_length=512)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, max_length=512)

In [ ]:
def predict_for_instruction(instruction, text, model):
    text = text.replace('\n', ' ')
    prompt = ("Below is an instruction that describes a task, paired with an input that provides further context. "
              "Write a response that appropriately completes the request.\n\n"
              f"### Instruction:\n{instruction}\n\n### Input:\n{text}\n\n### Response:")

    inputs = tokenizer([prompt],
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        # this parameters are also important but you can read about them in the docs and just try changing them
       # num_beams=1,
      #  temperature=0.4,
        max_length=1500,
        no_repeat_ngram_size=10,
        repetition_penalty= 5.0,
    #     length_penalty=0.01,
         early_stopping=True,
    #    do_sample=True,
    #     top_k=30,
    #     top_p=0.8,
     #   early_stopping=False,
    #     num_return_sequences=3,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=True)
    return summaries[0]

In [ ]:
def predict_for_instruction(instruction, text, model):
    text = text.replace('\n', ' ')
    prompt = ("Below is an instruction that describes a task, paired with an input that provides further context. "
              "Write a response that appropriately completes the request.\n\n"
              f"### Instruction:\n{instruction}\n\n### Input:\n{text}\n\n### Response:")

    inputs = tokenizer([prompt],
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        # this parameters are also important but you can read about them in the docs and just try changing them
        num_beams=1,
#         temperature=0.4,
         max_length=1000,
  #      max_new_tokens=80,
#         no_repeat_ngram_size=3,
    #     repetition_penalty= 5.0,
    #     length_penalty=0.01,
    #     early_stopping=True,
    #     do_sample=True,
    #     top_k=30,
    #     top_p=0.8,
        early_stopping=True,
    #     num_return_sequences=3,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=True,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=True)
    return summaries[0]

In [ ]:
text = """
Известное государственное слово подействовало на смотрителя усыпляюще. Он как бы
сразу заснул, перестав бормотать и оправдываться. Его слегка согнутая в пояснице фигура в
короткой душегрейке, плюшевых штанах и высоких белых, подшитых желтой кожей валенках застыла неподвижно в сумраке просторной, сильно натопленной горницы. Зато его жена, тихо до этого сидевшая с вязаньем в дальнем углу за ситцевой занавеской, заворочалась, выглянула, показывая свое широкое, ничего не выражающее лицо, уже успевшее осточертеть доктору за эти два часа ожидания, пития чая с малиновым и сливовым вареньем и листания прошлогодней «Нивы»"""

In [ ]:
instruction = "Сократи текст."
predict_for_instruction(instruction, text, model)

'На смотрителя усыпляюще подействовало на смотрителя подействовало, приведенно, что в смотрителя кожей подействовало находить высокие и исношения. Подавайтесь усыпью продлительным простом в давлении могут быть жизнь и отправительно, если вы оказ'

текст действительно сократился, но...

In [ ]:
text = """
"""

In [ ]:
instruction = "Расскажи о процессе написания хорошего резюме."
predict_for_instruction(instruction, text, model)

'Когда люди произведены, находиться опасное место? К вкусным приложением резюме, вам которые могут включать почты? К нашу перцу он был более достопримечательным.'

In [ ]:
text = """
∫(3x² - 2x + 1)dx
"""

In [ ]:
instruction = "Решите следующий интеграл:"
predict_for_instruction(instruction, text, model)

'x=y^2'

В целом модель ужасно бредит, но как будто к инструкциям пытается прислушаться))